# googleMapReviewPreprocessing
* 코드 목적: 텍스트 데이터를 모델 성능을 높일 수 있도록 정규표현식과 맞춤법 교정 라이브러리, 띄어쓰기 교정 라이브러리 등을 활용하여 데이터를 정제한다. 

## 초기 코드에서 발견된 문제점 
1. 맞춤법이 틀린 문장은 추후 토크나이징할 때 영향있을거 같음. => 맞춤법 교정, 띄어쓰기 교정을 추가함.
2. (Google 번역)처럼 괄호로 둘러쌓인 단어들이 '번역', '원문'과 같은 단어로 남아있음.
3. 코드 정리 필요
4. 전체 데이터셋에 대한 정제 데이터셋 만들어야함.

## 수정 후 발견된 문제점
1. dataframe에 keyword 다르게 매칭됨. 
2. 맞춤법 교정 후 데이터 전처리 안 해줌. 
3. 제대로 전처리가 진행됐는지 확인하는 코드 없음.
4. 단어를 빼먹음 => 정규표현식 문제였음. 수정 완료 
5. 완성형 한글만 남길 것 => 정규표현식 수정

## Data path 확인

In [61]:
!pwd # 현재 경로 확인 

/home/aiffel-dj19/jungcheck/DataPreprocessing


아래 tree 명령어로 상위 폴더(jungcheck)내 파일 구성을 볼 수 있습니다.    

In [62]:
!ls . # 현재 위치의 file list

 134963_norm.txt		     googleReviewsTextCloud.ipynb
'대전둘레산길 여행 대전.csv'	     labeled
 data				     naverReviewPreprocessingV0.ipynb
 filtered			     naverReviewPreprocessingV3.ipynb
 googleLabeling.ipynb		     reference
 googleReviewCompanion.ipynb	     soyspacing.model
 googleReviewPreprocessingV0.ipynb   space_rule.txt
 googleReviewPreprocessingV1.ipynb


In [63]:
!tree -L 3 -N ./data/google_review # jungcheck 폴더내 경로 확인 

./data/google_review
├── 01오월드
│   └── 오월드.csv
├── 02뿌리공원
│   └── 뿌리공원.csv
├── 03엑스포
│   ├── 한빛탑.csv
│   ├── 엑스포다리.csv
│   ├── 엑스포음악분수.csv
│   ├── 엑스포과학공원.csv
│   └── 세계엑스포기념품박물관.csv
├── 04한밭수목원
│   └── 한밭수목원.csv
├── 05계족산
│   ├── 계족산(봉우리읍내동).csv
│   ├── 계족산성.csv
│   └── 계족산황톳길.csv
├── 06대청호
│   ├── 대청호.csv
│   ├── 대청댐물문화관.csv
│   └── 대청호반자연생태공원.csv
├── 07장태산
│   ├── 장태산자연휴양림.csv
│   ├── 장태산자연휴양림전망대.csv
│   ├── 장태산자연휴양림숲속의집.csv
│   └── 장태산자연휴양림메타세콰이어산림욕장.csv
├── 09동춘당
│   ├── 회덕동춘당.csv
│   └── 동춘당공원.csv
├── 11으능정이문화의거리
│   ├── 으능정이거리.csv
│   ├── 대전스카이로드.csv
│   └── 으능정이문화거리.csv
├── 12유성온천
│   ├── 유성온천공원.csv
│   └── 유성온천족욕체험장.csv
├── 13성심당
│   ├── 성심당본점.csv
│   ├── 성심당대전역점.csv
│   ├── 성심당케잌부띠끄본점.csv
│   ├── 성심당롯데백화점대전점.csv
│   └── 성심당DCC점.csv
├── 13시민천문대
│   └── 대전시민천문대.csv
├── 14수통골
│   ├── 수통골유원지.csv
│   └── 계룡산국립공원수통골지구.csv
├── 16대전문화예술단지
│   ├── 이응노미술관.csv
│   ├── 대전예술의전당.csv
│   └── 대전시립미술관.csv
└── 17국립중앙과학관
    └── 국립중앙과학관.csv

15 directories, 37 files


## Library Import

In [64]:
import os 
import glob
from tqdm import tqdm

import re
import pandas as pd 
import copy # deep copy

from hanspell import spell_checker # 한글 맞춤법 검사기 
from hangul_checker import KoreanSpellChecker # 한글 맞춤법 검사기 
from pykospacing import Spacing # 한글 띄어쓰기 검사기 

## Data Load 

In [65]:
data_path = os.path.join(os.getcwd(), 'data/google_review/')
data_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/'

In [66]:
folders = os.listdir(data_path)
print(folders)
print(len(folders)) # 17개 대상 관광지 중 15개 

['07장태산', '05계족산', '09동춘당', '11으능정이문화의거리', '02뿌리공원', '03엑스포', '13성심당', '14수통골', '12유성온천', '04한밭수목원', '17국립중앙과학관', '06대청호', '16대전문화예술단지', '13시민천문대', '01오월드']
15


👉 17개 대상 관광지 중에 15개 데이터만 크롤링 해올 수 있었습니다. 구글 리뷰 같은 경우 한밭야구장과 둘레길은 리뷰가 없거나 2개 이하 였기 때문에 데이터셋에 포함되지 않습니다.    
👉 감성 분석 결과를 네이버 블로그 리뷰와 합쳐서 17개 대상 관광지들에 대한 감성 분석을 진행합니다.
👉 카이스트 같은 경우 크롤링 데이터셋에 있었지만, 대상 관광지에 포함되지 않아 코드 진행하기 전에 디렉토리에서 삭제했습니다.

In [67]:
# 대전관광 홈페이지 영문버전을 참고했으며(단, 긴 경우 줄임), 해당 리스트는 정제후 파일명으로 사용
# 나오지 않는 관광지는 네이버 검색 기준을 따름
keywords_ko = [file_name[2:] for file_name in folders]
keywords_en = ['jangtae_mountain', 'gyejok_mountain', 'dongchundang', 'uineungjeongi_street', 'ppuri_park', 
               'expo_science_park', 'sungsimdang_bakery', 'water_barrel', 'yuseong_hotspring', 'hanbat_arboretum', 
               'science_museum','daecheong_lake', 'art_culture_complex', 'observatory', 'oworld_zoo']

keywords = {k:v for k, v in zip(keywords_ko, keywords_en)}
keywords

{'장태산': 'jangtae_mountain',
 '계족산': 'gyejok_mountain',
 '동춘당': 'dongchundang',
 '으능정이문화의거리': 'uineungjeongi_street',
 '뿌리공원': 'ppuri_park',
 '엑스포': 'expo_science_park',
 '성심당': 'sungsimdang_bakery',
 '수통골': 'water_barrel',
 '유성온천': 'yuseong_hotspring',
 '한밭수목원': 'hanbat_arboretum',
 '국립중앙과학관': 'science_museum',
 '대청호': 'daecheong_lake',
 '대전문화예술단지': 'art_culture_complex',
 '시민천문대': 'observatory',
 '오월드': 'oworld_zoo'}

👉 대전관광 홈페이지 영문버전을 참고했습니다. (단, 긴 경우 줄임)    
👉 맨 뒤에 단어는 해당 관광지가 어떤 유형의 장소에 해당하는지를 나타내는 단어로 통일성을 줬습니다. 감성 분석을 할 때 성능이 잘 안 나오다면, 장소 유형이 같은 대상끼리 단어사전을 만들어주는 방법도 생각해볼 수 있습니다. 

In [68]:
if '장태산' in keywords.keys():
    print(keywords['장태산'])ㅋ

jangtae_mountain


In [69]:
# root_path내의 dataframe을 합해주는 함수
def concatCsv(root_path:str, keys:dict, folders_list:list):
    data_list = []
    
    for folder in folders_list:
        folder_path = os.path.join(root_path, folder)
        key_ko = folder[2:]
        
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            print(file_path)
            
            search = file.split('.')[0]               # 파일명에서 키워드만 추출    
            df = pd.read_csv(file_path, encoding='utf-8')  # csv 파일 읽기
            df['search'] = search
            
            if key_ko in keywords.keys():
                df['keyword'] = keywords[key_ko]
            data_list.append(df)
        print('-'*20)
    df = pd.concat(data_list, axis=0)

    return df

In [70]:
data_df = concatCsv(data_path, keywords, folders)

/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/07장태산/장태산자연휴양림메타세콰이어산림욕장.csv
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/07장태산/장태산자연휴양림.csv
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/07장태산/장태산자연휴양림전망대.csv
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/07장태산/장태산자연휴양림숲속의집.csv
--------------------
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/05계족산/계족산황톳길.csv
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/05계족산/계족산(봉우리읍내동).csv
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/05계족산/계족산성.csv
--------------------
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/09동춘당/회덕동춘당.csv
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/09동춘당/동춘당공원.csv
--------------------
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/11으능정이문화의거리/으능정이거리.csv
/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/11으능정이문화의거리/대전스카이로드.

In [71]:
data_df

,name,ratings,date,comment,search,keyword
0,미샤,4,2주 전,여기는 갈 때마다 새로운 느낌이야 즐거운 시간이 되었습니다.,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
1,Gyang gree young,5,3주 전,하늘보고 누워 힐링하기 넘 좋은곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
2,박진수,5,3달 전,잘 정리된 메타세쿼이아 숲에서 산림욕 하기 좋은 곳으로 둘레길 걷기와 전망대까지 가...,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
3,Sh Choi,5,7달 전,산책 하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
4,JY Jeong,4,8달 전,없음,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
...,...,...,...,...,...,...
925,김경영,4,1년 전,괜찮아요,오월드,oworld_zoo
926,패티가고,5,1년 전,없음,오월드,oworld_zoo
927,이종진,3,1년 전,없음,오월드,oworld_zoo
928,김소연,5,1년 전,겨울에도 놀기 좋아요~ㅎㅎ,오월드,oworld_zoo


In [72]:
type(data_df['keyword'].iloc[0])

str

In [73]:
# keyword 누락된 열 없는지 확인
data_df['keyword'].isnull().sum()

0

In [74]:
# keyword 구성 동일한지 확인 
sorted(keywords.values()) == sorted(list(set(data_df['keyword'])))

True

In [82]:
data_df['keyword'].value_counts()

sungsimdang_bakery      3800
art_culture_complex     2137
expo_science_park       1812
water_barrel            1473
daecheong_lake          1106
yuseong_hotspring        998
jangtae_mountain         995
uineungjeongi_street     977
gyejok_mountain          953
hanbat_arboretum         930
ppuri_park               930
science_museum           930
oworld_zoo               930
dongchundang             780
observatory               49
Name: keyword, dtype: int64

In [ ]:
# 위의 결과 시각화 => 보고서 그림0 

In [84]:
data_df['keyword'].value_counts().sum() / 15

1253.3333333333333

## Data cleaning
* null 처리 
* 일정 길이 이하의 문자열 제거 
* 중복 제거 

In [15]:
# null 처리
print(f'Before length: {len(data_df)}')

data_df = data_df[data_df['comment'] != '없음']
print(f'After length: {len(data_df)}')

Before length: 18800
After length: 8635


In [16]:
# 단어 1개인 경우 확인 
data_df[data_df['comment'].map(len) == 1]

,name,ratings,date,comment,search,keyword
63,채규삼,5,3주 전,ㆍ,장태산자연휴양림,jangtae_mountain
521,남홍계용,5,7달 전,😀,장태산자연휴양림,jangtae_mountain
732,hunyong lee,5,9달 전,츠,장태산자연휴양림,jangtae_mountain
929,전영옥,4,1년 전,ᆞ,장태산자연휴양림,jangtae_mountain
20,남LeeSoonChul,4,3달 전,😀,계족산성,gyejok_mountain
84,허현강,5,9달 전,굿,계족산성,gyejok_mountain
526,김상민,5,2년 전,굿,계족산성,gyejok_mountain
72,물방울,5,2달 전,굳,대전스카이로드,uineungjeongi_street
270,대전아트카튠,5,8달 전,굿,대전스카이로드,uineungjeongi_street
278,양성혁,4,9달 전,큼,대전스카이로드,uineungjeongi_street


👉 데이터를 확인했을때 유의미한 경우도 있기 때문에 length=1인 데이터 유지 

In [17]:
# 단어 0개인 comment 제외 
print(f'Before length: {len(data_df)}')

data_df = data_df[data_df['comment'].map(len) > 0] 
print(f'After length: {len(data_df)}')

Before length: 8635
After length: 8635


In [18]:
# 중복제거 
print(f'Before length: {len(data_df)}')

data_df = data_df.drop_duplicates(['name', 'keyword', 'date']) # name과 keyword, date 모두 중복일때 제거 
print(f'After length: {len(data_df)}')

Before length: 8635
After length: 8466


In [19]:
data_df

,name,ratings,date,comment,search,keyword
0,미샤,4,2주 전,여기는 갈 때마다 새로운 느낌이야 즐거운 시간이 되었습니다.,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
1,Gyang gree young,5,3주 전,하늘보고 누워 힐링하기 넘 좋은곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
2,박진수,5,3달 전,잘 정리된 메타세쿼이아 숲에서 산림욕 하기 좋은 곳으로 둘레길 걷기와 전망대까지 가...,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
3,Sh Choi,5,7달 전,산책 하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
6,박은선,5,1년 전,주말마자 애정하는 장태산나들이:-) 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
...,...,...,...,...,...,...
921,young wu Kwon,5,1년 전,재밌어요..😍😍,오월드,oworld_zoo
922,강강수월래,4,1년 전,애들놀고 구경거리는 굿,오월드,oworld_zoo
924,원언니,5,1년 전,초등생전용놀이공원으로 아담하고 이뻐요,오월드,oworld_zoo
925,김경영,4,1년 전,괜찮아요,오월드,oworld_zoo


## Data preprocessing

In [20]:
filtered_df = copy.deepcopy(data_df)

In [21]:
filtered_df['comment'][filtered_df['search']=='대전예술의전당'][:20]

0                                  대전시민을 위한 최대규모의 공연시설물
4     코로나때문에 너무 오랜만에 가본 예술의 전당이였고 공연도 너무 좋았습니다~\n직원분...
6                                 시설이 깨끗하고 음향이 좋은 곳입니다.
10    굿 클래식~^^\n코로나인데도 연주자들 안전하게 무대에 올려주셔서 기획자님께 고맙다...
11                                                    굿
14                          쾌적하고 깨끗하고 넓고 아주 좋은 장소입니다.^^
16                                                 깔끔해요
18                                         공연봤어요 크고 좋네유
21      넓직한 공간, 그냥 밖에 있어도 휴식 공간이 되고, 실내는 늘 품격있는 공연만 한다.
22                                            풍류 시노래콘서트
23                                 아침을 여는 클래식 너무 좋았습니다~
29                                         많은 공연들을 하는 곳
30    코로나 때문에 작년은 너무 유울했어요. 예술관람을 거의 못했거든요.. 이번에 대관공...
33                                 앙상블홀 공연 시작전\n베이스 연광철
35                                      무대와 객석이 너무 멀다..
36    뮤지컬을 관람하기위해 방문했는데 앞뒤 좌석 간격이 좁다는 생각이 들었음 앞사람이 조...
37                                  3층 봤는데 보는자리가 너무좁았어요
38    화장실은 가지 않았어요. 공연 관람 후 주차 요금은 받지 않았어요. 좋은 관

👉 데이터셋을 살펴보면, 외국인이 작성한 것도 한글로 번역되어 크롤링됐다.
따로 영어를 살려줄 필요 없어 보이므로 한글, ~숫자~ 제외하고 다 제거한다. 

In [22]:
# 번역, 원문과 같이 구글 번역에 의해 나타나는 단어를 제거하는 정규표현식 패턴
test = '안녕하 (번역 제공) 번역 원문 sdfsdf(구글 번여) 231231연연ㄴㄹㄴㄹㄴㅇㅇㄹ  번역 공연 안녕하'
test = re.sub("[번역 제공|번역|원문]", "", test) # 문제: 각 글자를 제거하는 대상으로 인식함
test = re.sub("\([^)]*\)", "", test) # 괄호 사이 단어 제거 
test = re.sub('[^가-힣|0-9]', "", test)
test

'안녕하231231연연연안녕하'

👉 정규표현식 테스트 

In [23]:
test_df = filtered_df.copy()

In [24]:
test_df['comment'] = test_df['comment'].str.replace("[^ㄱ-ㅎ|ㅏ-ㅣ]"," ", regex=True) # 완성형 한글만
test_df['comment'] = test_df['comment'].str.strip() # 공백 제거 
for i in test_df['comment']:
    if len(i) > 0:
        print(i)

ㅠㅠ
ㅡㅡ
ㅠㅠ
ㅋㅋㅋ
ㅅ
ㅎㅎ
ㅜㅜㅜ                                                   ㅎㅎㅎㅎ
ㅠㅠ
ㅠ
ㅇㅇ
ㅠㅠ
ㅡ
ㅎㅎ
ㅈ
ㅠ
ㅇ
ㅎ
ㅎㅎ
ㄱㄱㄱ
ㅎㅎ
ㅎㅎ
ㄹ
ㅋㅋㅋ
ㅇ
ㅋㅋ
ㅠㅠㅠ
ㅎㅎㅎㅎ
ㅣ
ㅎ
ㅎㅎ
ㅣ
ㅅ
ㅠ ㅠ
ㅎㅎ
ㅡㅡ
ㅎ
ㅈㅎ
ㅠㅠ
ㅎㅎ
ㅋㅋ
ㅠ
ㅠㅠ
ㅇ ㅁ
ㅎ
ㅋㅋㅋ
ㅎㅎ
ㅜㅜ
ㅡㅡ
ㅋ
ㅠㅠ
ㅎㅎ
ㅇ
ㅠ
ㅎ         ㅎ
ㅜㅜ
ㅋ
ㅋㅋ
ㅋㅋ
ㅡ            ㅡㅡ
ㅋㅅㅋ                   ㅋ
ㅁ
ㅎ
ㅋ
ㅡ
ㅎㅎ
ㅎㅎ
ㅎ                   ㅎ
ㅠㅜ
ㅎ
ㅏ
ㅋ
ㅠㅠ
ㅠ ㅠ
ㅎㅎ
ㅎ
ㅎㅎ
ㅋ
ㅡ
ㅎㅎㅎ
ㅡ ㅡ
ㅎㅎ
ㅎ
ㅋ
ㅋㅋㅋ
ㅎ
ㅎ
ㅎㅎ
ㅋㅋ
ㅠㅠㅠㅠㅠ
ㅠ ㅠ
ㅓ
ㅎㅎ
ㅠㅠ                               ㅠㅠ
ㅋ
ㅎㅎ
ㅋ                ㅎㅎ                   ㅎㅎ
ㅎㅎ
ㅎ
ㅎ
ㅠ ㅜ
ㅠㅠ
ㅎ
ㅜㅜ                                                        ㅡ
ㅋ
ㅠㅜㅠ
ㅜ
ㅎ
ㅠㅠ
ㅎㅎㅎ
ㅜㅜ
ㅠ
ㅡㅡ
ㅎ
ㅎㅎ
ㅎㅎ
ㅠ
ㅠ
ㅠㅠ
ㅠ
ㅠㅠ
ㅍㄹㅂㄱㅌ
ㅎㅎ
ㅎㅎ                                                                                                              ㅎㅎ
ㅠㅠ
ㅋㅋ
ㅋ
ㅅ
ㅠㅠ
ㅠ
ㅜㅜ
ㅠ
ㅎ
ㅋㅋ
ㅎㅎ
ㅎ
ㅋㅋㅋㅋㅋㅋ
ㅋㅋㅋ
ㅋㅋ
ㅎ
ㅜㅜ
ㅡ
ㅜㅜ                ㅜㅜ
ㅠㅠ                                                          ㅠㅠㅠ                                                                        ㅜ
ㅋㅋ
|                                                                                    

👉 비완성형 한글만 남겼을때 결과를 보면 ㅋㅋㅋ , ㅎㅎ 등 의미없는 데이터들이 위주이기 때문에 정규표현식에서는 완성형 한글만 취급한다. 

In [25]:
# filtered_df['comment'] = filtered_df['comment'].str.replace("[(].*?[)]","", regex=True) # 괄호 사이 단어 제거
filtered_df['comment'] = filtered_df['comment'].str.replace("\([^)]*\)"," ", regex=True) # 위에 정규표현식 적용안되서 다시 적용
filtered_df['comment'] = filtered_df['comment'].str.replace("[^\w\s]"," ", regex=True)
# filtered_df['comment'] = filtered_df['comment'].str.replace("[^가-힣|0-9|ㄱ-ㅎ|ㅏ-ㅣ]"," ", regex=True) # 완성형 한글만
filtered_df['comment'] = filtered_df['comment'].str.replace("[^가-힣|0-9]"," ", regex=True) # 완성형 한글만
# filtered_df['comment'] = filtered_df['comment'].str.replace(" +"," ", regex=True)
filtered_df['comment'] = filtered_df['comment'].str.strip()
filtered_df['comment'] = filtered_df['comment'].apply(lambda row: " ".join(row.split())) # 정규표현식보다 빠름.. 
filtered_df

,name,ratings,date,comment,search,keyword
0,미샤,4,2주 전,여기는 갈 때마다 새로운 느낌이야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
1,Gyang gree young,5,3주 전,하늘보고 누워 힐링하기 넘 좋은곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
2,박진수,5,3달 전,잘 정리된 메타세쿼이아 숲에서 산림욕 하기 좋은 곳으로 둘레길 걷기와 전망대까지 가...,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
3,Sh Choi,5,7달 전,산책 하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
6,박은선,5,1년 전,주말마자 애정하는 장태산나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
...,...,...,...,...,...,...
921,young wu Kwon,5,1년 전,재밌어요,오월드,oworld_zoo
922,강강수월래,4,1년 전,애들놀고 구경거리는 굿,오월드,oworld_zoo
924,원언니,5,1년 전,초등생전용놀이공원으로 아담하고 이뻐요,오월드,oworld_zoo
925,김경영,4,1년 전,괜찮아요,오월드,oworld_zoo


👉 정규표현식의 Pattern은 단어가 아니라 한 글자씩 해당하기 때문에 계속 공연-> 연 등으로 단어가 잘리는 문제가 있었던 거였습니다.     
해당 문제를 해결했고 이후 원문, 구글, 번역 등의 결과가 있는지 확인했을떄 구글 아트 앤 컬쳐라는 이응노 미술관에서 실제 있었던 전시회 사례 이외에는 나오지 않음을 확인했습니다. 

In [26]:
for i in range(len(filtered_df)):
    if '원문' in filtered_df['comment'].iloc[i]:
        print(filtered_df['comment'].iloc[i])
    if '구글' in filtered_df['comment'].iloc[i]:
        print(filtered_df['comment'].iloc[i])
    if '번역' in filtered_df['comment'].iloc[i]:
        print(filtered_df['comment'].iloc[i])

구글 아트 앤 컬쳐와 협업해서 미디어로 이응노 대표작품들을 만날 수 있습니다
이응노 미술관 사전예약 안해서 못 보고 돌아왔어욤 대전 시립미술관 관람 이응노와 구글 아트앤컬처 10월4일 까지 전시하니 가실 분들 사전 예약 필수 꼭 하고 가세요


👉 정규표현식을 통해 제거 후에, 문장 구성이 달라지기 때문에 cleaning 작업을 위와 똑같이 다시 진행해줍니다.

In [27]:
# 단어 1개인 경우 확인 
filtered_df[filtered_df['comment'].map(len) == 1]

,name,ratings,date,comment,search,keyword
695,Soo Young Yoo,5,8달 전,굿,장태산자연휴양림,jangtae_mountain
732,hunyong lee,5,9달 전,츠,장태산자연휴양림,jangtae_mountain
84,허현강,5,9달 전,굿,계족산성,gyejok_mountain
324,Soo Young Yoo,5,1년 전,굿,계족산성,gyejok_mountain
526,김상민,5,2년 전,굿,계족산성,gyejok_mountain
...,...,...,...,...,...,...
403,오진우,5,5달 전,잼,오월드,oworld_zoo
428,류라히,5,6달 전,굿,오월드,oworld_zoo
633,daniel. ggl,4,8달 전,굿,오월드,oworld_zoo
693,손장현,5,10달 전,굿,오월드,oworld_zoo


In [28]:
# 단어 0개인 comment 제외 
print(f'Before length: {len(filtered_df)}')

filtered_df = filtered_df[filtered_df['comment'].map(len) > 0] 
print(f'After length: {len(filtered_df)}')

Before length: 8466
After length: 8428


In [29]:
# 중복제거 
print(f'Before length: {len(filtered_df)}')

filtered_df = filtered_df.drop_duplicates(['name', 'keyword', 'date']) # name과 keyword, date 모두 중복일때 제거 
print(f'After length: {len(filtered_df)}')

Before length: 8428
After length: 8428


In [30]:
filtered_df

,name,ratings,date,comment,search,keyword
0,미샤,4,2주 전,여기는 갈 때마다 새로운 느낌이야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
1,Gyang gree young,5,3주 전,하늘보고 누워 힐링하기 넘 좋은곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
2,박진수,5,3달 전,잘 정리된 메타세쿼이아 숲에서 산림욕 하기 좋은 곳으로 둘레길 걷기와 전망대까지 가...,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
3,Sh Choi,5,7달 전,산책 하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
6,박은선,5,1년 전,주말마자 애정하는 장태산나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
...,...,...,...,...,...,...
921,young wu Kwon,5,1년 전,재밌어요,오월드,oworld_zoo
922,강강수월래,4,1년 전,애들놀고 구경거리는 굿,오월드,oworld_zoo
924,원언니,5,1년 전,초등생전용놀이공원으로 아담하고 이뻐요,오월드,oworld_zoo
925,김경영,4,1년 전,괜찮아요,오월드,oworld_zoo


## 맞춤법 교정

In [31]:
def spacingCorrect(sen):
    spacing = Spacing(rules=keywords_ko+['이뻐요'])
#     print(f'Before sentence: {sen}')

    sen = spacing(sen)
#     print(f'After sentence: {sen}')
#     print('-'*40)
    
    return sen

In [32]:
def spellCorrect(sen):
#     ksc = KoreanSpellChecker()
#     print(f'Before sentence: {sen}')

#     sen = ksc.check_spelling(sen)
#     print(f'After sentence: {sen}')
#     print('-'*40)
    result = spell_checker.check(sen)
    sen = result.as_dict()['checked']
    return sen

In [33]:
print(f'Before length: {len(filtered_df)}')

Before length: 8428


In [34]:
# # # 4분 34초 소요
filtered_df['comment'] = [spacingCorrect(sen) for sen in tqdm(filtered_df['comment'])]

100%|██████████| 8428/8428 [04:41<00:00, 29.98it/s]


In [35]:
print(f'After length: {len(filtered_df)}')

After length: 8428


In [36]:
filtered_df

,name,ratings,date,comment,search,keyword
0,미샤,4,2주 전,여기는 갈 때마다 새로운 느낌이야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
1,Gyang gree young,5,3주 전,하늘 보고 누워 힐링하기 넘 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
2,박진수,5,3달 전,잘 정리된 메타세쿼이아 숲에서 산림욕 하기 좋은 곳으로 둘레길 걷기와 전망대까지 가...,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
3,Sh Choi,5,7달 전,산책 하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
6,박은선,5,1년 전,주말 마자 애정하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
...,...,...,...,...,...,...
921,young wu Kwon,5,1년 전,재밌어요,오월드,oworld_zoo
922,강강수월래,4,1년 전,애들 놀고 구경거리는 굿,오월드,oworld_zoo
924,원언니,5,1년 전,초등생 전용 놀이공원으로 아담하고 이뻐요,오월드,oworld_zoo
925,김경영,4,1년 전,괜찮아요,오월드,oworld_zoo


In [37]:
# print(f'Before length: {len(filtered_df)}')

👉 KoreanSpellChecker를 사용할 경우, 맞춤법 검사기의 최대 허용 글자수는 500자이므로, 글자수 제한을 둬야합니다.

In [38]:
# KoreanSpellChecker
# filtered_df = filtered_df[filtered_df['comment'].map(len) <= 500] # name과 keyword, date 모두 중복일때 제거
# filtered_df

In [39]:
# # # # Hanspell - 7분 4초 정도 소요
# filtered_df['comment'] = [spellCorrect(sen) for sen in tqdm(filtered_df['comment'])]

In [40]:
# print(f'After length: {len(filtered_df)}')

In [41]:
# # set pandas option 
# pd.set_option('display.max_rows', None) # 전체 행 보기
# pd.set_option('display.max_columns', None) # 전체 열 보기 
# pd.set_option('display.expand_frame_repr', False)

In [42]:
# 단어 0개인 comment 제외 
print(f'Before length: {len(filtered_df)}')

filtered_df = filtered_df[filtered_df['comment'].map(len) > 0] 
print(f'After length: {len(filtered_df)}')

Before length: 8428
After length: 8428


In [43]:
# 중복제거 
print(f'Before length: {len(filtered_df)}')

filtered_df = filtered_df.drop_duplicates(['name', 'keyword', 'date']) # name과 keyword, date 모두 중복일때 제거 
print(f'After length: {len(filtered_df)}')

Before length: 8428
After length: 8428


In [44]:
# 단어 1개인 경우 comment값 확인 
one_length = filtered_df[filtered_df['comment'].map(len) == 1]['comment']
one_length = list(set(one_length))
one_length

['큰', '뿡', '잼', '큼', '네', '음', '빵', '앙', '츠', '굿', '짱', '굳']

In [60]:
# 단어 2개인 경우 comment값 확인 
two_length = filtered_df[filtered_df['comment'].map(len) == 2]['comment']
two_length = list(set(two_length))
two_length

['최애',
 '무료',
 '예쁜',
 '우우',
 '좋앙',
 '굿뜨',
 '아늑',
 '청결',
 '군상',
 '이쁨',
 '별로',
 '보통',
 '존멋',
 '평범',
 '좋아',
 '굿샤',
 '상쾌',
 '나름',
 '케익',
 '그저',
 '행복',
 '산책',
 '짱짱',
 '맛나',
 '쏘쏘',
 '선물',
 '이른',
 '선책',
 '좋음',
 '빵집',
 '공룡',
 '와와',
 '조음',
 '아유',
 '별루',
 '변비',
 '꿀맛',
 '괜츈',
 '맛집',
 '흙길',
 '더는',
 '족욕',
 '그냥',
 '쑈쑈',
 '힐링',
 '공원',
 '굿근',
 '맛남',
 '일출',
 '명소',
 '몰름',
 '구웃',
 '멎쁨',
 '멋짐',
 '넓다',
 '쵝오',
 '괜춘',
 '심한',
 '와우',
 '좋은',
 '노잼',
 '굳귣',
 '이뻐',
 '재다',
 '최기',
 '한적',
 '최고',
 '굳굳',
 '개꿀',
 '소소',
 '야경',
 '멎짐',
 '대전',
 '대박',
 '지림',
 '굿굿',
 '추천',
 '좋다',
 '깔끔',
 '좋지',
 '조녜',
 '전망',
 '맨위',
 '갠춘',
 '수업',
 '역쉬',
 '만족',
 '잼씀',
 '동네']

👉 한 글자로 된 단어 중에서는 '잼', '짱', '굳', '굿', '큼', '큰' 외에는 리뷰로 볼 수 없어 그 외는 삭제합니다. 

In [45]:
def oneLength(row: str):
    one_length = ['잼', '짱', '굳', '굿', '큼', '음'] 

    if len(row)==1 and row in one_length: return row
    elif len(row)==1 and row not in one_length: return ""
    else: return row

In [46]:
filtered_df['comment'] = filtered_df['comment'].apply(oneLength)
filtered_df

,name,ratings,date,comment,search,keyword
0,미샤,4,2주 전,여기는 갈 때마다 새로운 느낌이야 즐거운 시간이 되었습니다,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
1,Gyang gree young,5,3주 전,하늘 보고 누워 힐링하기 넘 좋은 곳이에요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
2,박진수,5,3달 전,잘 정리된 메타세쿼이아 숲에서 산림욕 하기 좋은 곳으로 둘레길 걷기와 전망대까지 가...,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
3,Sh Choi,5,7달 전,산책 하기 너무 좋은 명소,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
6,박은선,5,1년 전,주말 마자 애정하는 장태산 나들이 단풍도 예뻐요,장태산자연휴양림메타세콰이어산림욕장,jangtae_mountain
...,...,...,...,...,...,...
921,young wu Kwon,5,1년 전,재밌어요,오월드,oworld_zoo
922,강강수월래,4,1년 전,애들 놀고 구경거리는 굿,오월드,oworld_zoo
924,원언니,5,1년 전,초등생 전용 놀이공원으로 아담하고 이뻐요,오월드,oworld_zoo
925,김경영,4,1년 전,괜찮아요,오월드,oworld_zoo


In [47]:
# 단어 0개인 comment 제외 
print(f'Before length: {len(filtered_df)}')

filtered_df = filtered_df[filtered_df['comment'].map(len) > 0] 
print(f'After length: {len(filtered_df)}')

Before length: 8428
After length: 8421


In [48]:
# 전체 keywords가 다 포함됐는지 확인
sorted(keywords.values()) == sorted(list(set(filtered_df['keyword'])))

True

In [49]:
filtered_df[filtered_df['keyword']=='oworld_zoo']['search'].count()

446

In [50]:
for _, key_en in keywords.items():
    cnt = filtered_df[filtered_df['keyword']==key_en]['search'].count()
    print(f'{key_en}과 연관된 검색어는 {cnt}개')
    print(list(set(filtered_df[filtered_df['keyword']==key_en]['search'])))
    print('-'*40)

jangtae_mountain과 연관된 검색어는 518개
['장태산자연휴양림메타세콰이어산림욕장', '장태산자연휴양림', '장태산자연휴양림전망대', '장태산자연휴양림숲속의집']
----------------------------------------
gyejok_mountain과 연관된 검색어는 483개
['계족산(봉우리읍내동)', '계족산성', '계족산황톳길']
----------------------------------------
dongchundang과 연관된 검색어는 329개
['동춘당공원', '회덕동춘당']
----------------------------------------
uineungjeongi_street과 연관된 검색어는 333개
['대전스카이로드', '으능정이문화거리', '으능정이거리']
----------------------------------------
ppuri_park과 연관된 검색어는 426개
['뿌리공원']
----------------------------------------
expo_science_park과 연관된 검색어는 784개
['엑스포과학공원', '엑스포다리', '한빛탑', '엑스포음악분수', '세계엑스포기념품박물관']
----------------------------------------
sungsimdang_bakery과 연관된 검색어는 1863개
['성심당케잌부띠끄본점', '성심당롯데백화점대전점', '성심당본점', '성심당대전역점', '성심당DCC점']
----------------------------------------
water_barrel과 연관된 검색어는 685개
['계룡산국립공원수통골지구', '수통골유원지']
----------------------------------------
yuseong_hotspring과 연관된 검색어는 431개
['유성온천공원', '유성온천족욕체험장']
----------------------------------------
hanbat_arboretum과 연관된

👉 검색어에 해당하는 search 컬럼을 확인하여 keyword와 매칭이 잘 됐는지 확인했습니다. 확인한 결과, keyword와 search가 잘 매칭되었음을 확인할 수 있었습니다. 

## Save csv

In [51]:
import os
import pandas as pd

In [52]:
!pwd
!mkdir -p ./filtered/google_review/spellcheck
!mkdir -p ./filtered/google_review/spacing

/home/aiffel-dj19/jungcheck/DataPreprocessing


In [53]:
ls ./filtered/google_review

google_reviews_labeled_8429.csv  spacing_8431/             spellcheck_8420/
google_reviews_nolabel_8429.csv  spacing_spellcheck_8420/  spellcheck_8429/
spacing/                         spellcheck/


In [54]:
filtered_path = os.path.join(os.getcwd(), 'filtered/google_review/')
filtered_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/'

In [55]:
keywords

{'장태산': 'jangtae_mountain',
 '계족산': 'gyejok_mountain',
 '동춘당': 'dongchundang',
 '으능정이문화의거리': 'uineungjeongi_street',
 '뿌리공원': 'ppuri_park',
 '엑스포': 'expo_science_park',
 '성심당': 'sungsimdang_bakery',
 '수통골': 'water_barrel',
 '유성온천': 'yuseong_hotspring',
 '한밭수목원': 'hanbat_arboretum',
 '국립중앙과학관': 'science_museum',
 '대청호': 'daecheong_lake',
 '대전문화예술단지': 'art_culture_complex',
 '시민천문대': 'observatory',
 '오월드': 'oworld_zoo'}

In [56]:
# keyword별로 dataframe 저장 
def saveCsv(root_path: str, keys: dict, df):
    for _, key_en in keys.items():
        result_df = df[df['keyword']==key_en] 
        save_path = root_path + f'{key_en}_spacing.csv'
        result_df.to_csv(save_path, header=True, index=False)

In [57]:
saveCsv(filtered_path, keywords, filtered_df)

In [58]:
ls ./filtered/google_review/ # 파일 저장 경로내 파일 리스트 확인

art_culture_complex_spacing.csv  science_museum_spacing.csv
daecheong_lake_spacing.csv       spacing/
dongchundang_spacing.csv         spacing_8431/
expo_science_park_spacing.csv    spacing_spellcheck_8420/
google_reviews_labeled_8429.csv  spellcheck/
google_reviews_nolabel_8429.csv  spellcheck_8420/
gyejok_mountain_spacing.csv      spellcheck_8429/
hanbat_arboretum_spacing.csv     sungsimdang_bakery_spacing.csv
jangtae_mountain_spacing.csv     uineungjeongi_street_spacing.csv
observatory_spacing.csv          water_barrel_spacing.csv
oworld_zoo_spacing.csv           yuseong_hotspring_spacing.csv
ppuri_park_spacing.csv
